In [5]:
from project.models import Project
import pandas as pd
from django.db.models import F, Value, CharField, Q, Sum
from django.db.models.functions import Concat
from public_data.models import CommunePop, CommuneDiff, Scot

project = Project.objects.get(id=1486)
project.name

'Diagnostic de CA Grand Auch Coeur de Gascogne'

In [6]:
config = [
    {"name": "Insee", "db": F("city__insee"), "type": "index"},
    {"name": "Commune", "db": F("city__name"), "type": "index"},
    {"name": "EPCI", "db": F("city__epci__name"), "type": "index"},
    {"name": "SCoT", "db": F("city__scot__name"), "type": "index"},
    {
        "name": "Département",
        "db": F("city__departement__name"),
        "type": "index",
    },
    {
        "name": "Région",
        "db": F("city__departement__region__name"),
        "type": "index",
    },
    {
        "name": "Plus_ancien_millésime",
        "db": Value(project.first_year_ocsge),
        "type": "index",
    },
    {
        "name": "Plus_récent_millésime",
        "db": Value(project.last_year_ocsge),
        "type": "index",
    },
    {
        "name": "Différentiel",
        "db": Value(project.last_year_ocsge),
        "type": "columns",
    },
]
config

[{'name': 'Insee', 'db': F(city__insee), 'type': 'index'},
 {'name': 'Commune', 'db': F(city__name), 'type': 'index'},
 {'name': 'EPCI', 'db': F(city__epci__name), 'type': 'index'},
 {'name': 'SCoT', 'db': F(city__scot__name), 'type': 'index'},
 {'name': 'Département', 'db': F(city__departement__name), 'type': 'index'},
 {'name': 'Région', 'db': F(city__departement__region__name), 'type': 'index'},
 {'name': 'Plus_ancien_millésime', 'db': Value(2016), 'type': 'index'},
 {'name': 'Plus_récent_millésime', 'db': Value(2019), 'type': 'index'},
 {'name': 'Différentiel', 'db': Value(2019), 'type': 'columns'}]

In [7]:
qs = (
    CommuneSol.objects.filter(
        city__in=project.cities.all(), matrix__is_artificial=True, year=project.last_year_ocsge
    )
    .annotate(**{_["name"]: _["db"] for _ in config})
    .values(*[_["name"] for _ in config])
    .annotate(Surface_artificielle_dernier_millésime=Sum("surface"))
    .order_by("Insee")
)
qs.count()

34

In [11]:
df = pd.DataFrame(qs).fillna(0.0).set_index([_["name"] for _ in config])
df

,,,,,,,,,Surface_artificielle_dernier_millésime
Insee,Commune,EPCI,SCoT,Département,Région,Plus_ancien_millésime,Plus_récent_millésime,Différentiel,
32003,Antras,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2019,17.3000
32013,Auch,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2019,1702.3100
32014,Augnax,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2019,24.2800
32019,Auterive,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2019,93.0400
32024,Ayguetinte,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2019,39.5100
32054,Biran,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2019,127.8400
32059,Bonas,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2019,67.4800
32076,Castelnau-Barbarens,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2019,157.2300
32083,Castéra-Verduzan,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2019,174.5200


In [15]:
config[8]["db"] = Concat(
    "year_old", Value("-"), "year_new", output_field=CharField()
)

In [16]:
qs2 = (
    CommuneDiff.objects.filter(
        city__in=project.cities.all(),
        year_old__gte=project.analyse_start_date,
        year_old__lte=project.analyse_end_date,
    )
    .annotate(
        **{_["name"]: _["db"] for _ in config},
        Artificialisation=F("new_artif"),
        Renaturation=F("new_natural"),
        Artificialisation_nette=F("net_artif"),
    )
    .values(
        *[_["name"] for _ in config],
        "Artificialisation",
        "Renaturation",
        "Artificialisation_nette",
    )
)
qs2.count()

34

In [17]:
df2 = (
    pd.DataFrame(qs2)
    .fillna(0.0)
    .set_index([_["name"] for _ in config])
)
df2

,,,,,,,,,Artificialisation,Renaturation,Artificialisation_nette
Insee,Commune,EPCI,SCoT,Département,Région,Plus_ancien_millésime,Plus_récent_millésime,Différentiel,,,
32207,Leboulin,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2016-2019,0.3900,0.0200,0.3700
32014,Augnax,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2016-2019,1.2700,0.0600,1.2100
32089,Castillon-Massas,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2016-2019,0.0000,0.0000,0.0000
32453,Tourrenquets,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2016-2019,0.0000,0.0000,0.0000
32204,Lavardens,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2016-2019,0.5900,0.0000,0.5900
32091,Castin,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2016-2019,0.0000,0.0900,-0.0900
32183,Lahitte,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2016-2019,0.0000,0.0200,-0.0200
32382,Saint-Jean-Poutge,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2016-2019,0.8500,0.0000,0.8500
32083,Castéra-Verduzan,CA Grand Auch Coeur de Gascogne,De gascogne,Gers,Occitanie,2016,2019,2016-2019,3.5100,0.0000,3.5100


In [18]:
df3 = df.merge(df2, left_index=True, right_index=True)
df3

,,,,,,,,,Surface_artificielle_dernier_millésime,Artificialisation,Renaturation,Artificialisation_nette
Insee,Commune,EPCI,SCoT,Département,Région,Plus_ancien_millésime,Plus_récent_millésime,Différentiel,,,,
